# <font style = "color:rgb(139,0,0)">Neural Networks : Project 1 - Part 3</font> 

• CONTEXT: Develop a clickable GUI [desk application or web service application] which can automate Part I & II of this
project.

In [2]:
# Importing necessary libraries
import tkinter as tk
from tkinter import ttk
import pandas as pd
import re

# Creating application window
win = tk.Tk()
#  giving title to the window
win.title('Neural-Networks-GUI') 

# A:- Importing name of the dataframe

Name=ttk.Label(win,text="    Step 1: File Name")
Name.grid(row=0,column=0,sticky=tk.W)

Name_var=tk.StringVar()
Name_entrybox=ttk.Entry(win,width=16,textvariable=Name_var)
Name_entrybox.grid(row=0,column=1)

def Import_Data():
    global DB
    DF_Name=Name_var.get()
    DB_extension=re.findall("\..*", DF_Name) 
    if DB_extension==['.xlsx']:
        DB=pd.read_excel(DF_Name)
    elif DB_extension==['.csv']:
        DB=pd.read_csv(DF_Name)
# creating blank empty window to show the confirmation
    confirm="Done"
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=0,column=3)
    Confirm_entrybox.insert(1,str(confirm))   

Import_Data_Button=ttk.Button(win,text="Import Data",command=Import_Data)
Import_Data_Button.grid(row=0,column=2)


# B:- creating name of the target dataframe 

Target=ttk.Label(win,text="    Step 2: Target Colummn")
Target.grid(row=1,column=0,sticky=tk.W)

Target_var=tk.StringVar()
Target_entrybox=ttk.Entry(win,width=16,textvariable=Target_var)
Target_entrybox.grid(row=1,column=1)

# creating function for showing status
def Target_Data():
    global DB,X,y, Target_Name, X_train, X_test, y_train, y_test
    Target_Name=Target_var.get()
    
    Column_name=DB.columns
    Column_name
    found=0

    for i in range(len(Column_name)):
        if Column_name[i]==Target_Name:
            confirm="Found"
        else:
            confirm="Not Found"
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=1,column=3)
    Confirm_entrybox.insert(1,str(confirm))


Target_Button=ttk.Button(win,text="Import Target",command=Target_Data)
Target_Button.grid(row=1,column=2)


# C:- building regression model 

Modelling=ttk.Label(win,text="    Step 3: Neural Network Regressor")
Modelling.grid(row=2,column=0,sticky=tk.W)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential # Forward prop
from keras.layers import Dense, Activation, LeakyReLU
from keras import optimizers

Regression=ttk.Label(win,text="Regression")
Regression.grid(row=3,column=0,sticky=tk.E)


def NNReg():
    global DB,X,Y,NN_model_Regressor
    
    # Regression
    X=DB.drop('Signal_Strength',axis=1)     
    Y=DB['Signal_Strength']               

    X_Train,X_Test,Y_Train,Y_Test=train_test_split(X, Y, train_size=0.7, random_state=12)
    
    X_Train_S = StandardScaler().fit_transform(X_Train)   
    X_Test_S = StandardScaler().fit_transform(X_Test)     
    
    # Model 
    NN_model_Regressor = Sequential()
    NN_model_Regressor.add(Dense(128, kernel_initializer='normal',input_dim = X_Train.shape[1], activation='relu'))
    NN_model_Regressor.add(Dense(64, kernel_initializer='normal',activation='relu'))  # sigmoid, tanh
    NN_model_Regressor.add(Dense(32, kernel_initializer='normal'))
    NN_model_Regressor.add(LeakyReLU(alpha=0.1))
    NN_model_Regressor.add(Dense(16, kernel_initializer='normal'))
    NN_model_Regressor.add(LeakyReLU(alpha=0.1))
    NN_model_Regressor.add(Dense(1, kernel_initializer='normal'))  
    NN_model_Regressor.add(LeakyReLU(alpha=0.1))
    # Compile the network :
    NN_model_Regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
    NN_model_Regressor.summary()
    EPOCH=400
    Network_Regressor=NN_model_Regressor.fit(X_Train_S, Y_Train, validation_data=(X_Test_S,Y_Test), epochs=EPOCH, batch_size=200)
    
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=3,column=2)
    Confirm_entrybox.insert(1,str("Network Trained"))

Reg_Button=ttk.Button(win,text="Train",command=NNReg)
Reg_Button.grid(row=3,column=1)

from keras.models import model_from_json
import numpy
import os

Pickle=ttk.Label(win,text="Pickle")
Pickle.grid(row=4,column=0,sticky=tk.E)

def Pickle():
    # Pickling the model to JSON
    Regressor_model_json = NN_model_Regressor.to_json()
    with open("Regressor_model.json", "w") as json_file:
        json_file.write(Regressor_model_json)
    # Pickling weights to HDF5
    NN_model_Regressor.save_weights("Regressor_model.h5")
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=4,column=2)
    Confirm_entrybox.insert(1,str("Saved model to disk"))

Pickle_Button=ttk.Button(win,text="Run",command=Pickle)
Pickle_Button.grid(row=4,column=1)


# D:- Classifier 

Modelling=ttk.Label(win,text="    Step 4: Neural Network Classifier")
Modelling.grid(row=5,column=0,sticky=tk.W)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from keras.models import Sequential # Forward prop
from keras.layers import Dense, Activation, LeakyReLU
from keras import optimizers
from keras.models import model_from_json
import numpy
import os

Classifier=ttk.Label(win,text="Classifier")
Classifier.grid(row=6,column=0,sticky=tk.E)

def NNClassi():
    global DB,X,Y,NN_model_Classifier
    
    # Regression
    X=DB.drop('Signal_Strength',axis=1)   
    Y=DB['Signal_Strength']              

    X_Train,X_Test,Y_Train,Y_Test=train_test_split(X, Y, train_size=0.7, random_state=12)

    X_Train,X_Test,Y_Train,Y_Test=train_test_split(X, Y, train_size=0.7, random_state=12)
    
    X_Train_S = StandardScaler().fit_transform(X_Train)  
    X_Test_S = StandardScaler().fit_transform(X_Test)     
    
    Y_Train = to_categorical(Y_Train)
    Y_Test = to_categorical(Y_Test)
    
    NN_model_Classifier = Sequential()

    # The Input Layer :
    NN_model_Classifier.add(Dense(128, kernel_initializer='normal',input_dim = X_Train.shape[1], activation='relu'))

    # The Hidden Layers :
    NN_model_Classifier.add(Dense(64, kernel_initializer='normal',activation='relu'))  # sigmoid, tanh

    NN_model_Classifier.add(Dense(32, kernel_initializer='normal'))
    NN_model_Classifier.add(LeakyReLU(alpha=0.1))

    NN_model_Classifier.add(Dense(16, kernel_initializer='normal'))
    NN_model_Classifier.add(LeakyReLU(alpha=0.1))


    # The Output Layer :
    NN_model_Classifier.add(Dense(9, kernel_initializer='normal',activation='softmax'))  # except softmax

    # Compiling the network :
    NN_model_Classifier.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
    NN_model_Classifier.summary()
    EPOCH=400
    Network_Classifier=NN_model_Classifier.fit(X_Train_S, Y_Train, validation_data=(X_Test_S,Y_Test), epochs=EPOCH, batch_size=200)
    
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=6,column=2)
    Confirm_entrybox.insert(1,str("Network Trained"))

Cla_Button=ttk.Button(win,text="Train",command=NNClassi)
Cla_Button.grid(row=6,column=1)

Pickle=ttk.Label(win,text="Pickle")
Pickle.grid(row=4,column=0,sticky=tk.E)

def Pickle2():
    # Pickling the model to JSON
    Classifier_model_json = NN_model_Classifier.to_json()
    with open("Classifier_model.json", "w") as json_file:
        json_file.write(Classifier_model_json)
    # Pickling the weights to HDF5
    NN_model_Classifier.save_weights("Classifier_model.h5")
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=7,column=2)
    Confirm_entrybox.insert(1,str("Saved model to disk"))

Pickle_Button2=ttk.Button(win,text="Run",command=Pickle2)
Pickle_Button2.grid(row=7,column=1)

win.mainloop()